In [2]:
import pandas as pd

In [14]:
#df = pd.read_clipboard(decimal=",")

In [15]:
#df.to_pickle("mks_20220601.pkl")

In [16]:
df = pd.read_pickle("nichtkum_20220601.pkl")

In [17]:
df_scans = pd.read_pickle("scans_20220601.pkl")

In [18]:
df_mks = pd.read_pickle("mks_20220601.pkl")

In [21]:
df_mks.Key.value_counts()

Muss    705
Kann    490
Soll    394
Name: Key, dtype: int64

# Sonderauswertung

## Artikel Ø pro TAg

In [6]:
def average_articles_per_day(df, date_col="Datum"):
    list_with_articlenumber = []
    for day in df[date_col].unique():
        df_ = df[df[date_col] == day]
        list_with_articlenumber.append(df_.shape[0])
    average = np.mean(list_with_articlenumber)
    print(list_with_articlenumber)
    return average


print(average_articles_per_day(df))
print(f'Unit-Test: {df.groupby("Datum").agg({"ArtikelId":"count"}).mean()}.')

[71, 70, 78, 68, 108, 86, 93, 68, 69, 71, 91, 75, 76, 73, 105, 70, 85, 83, 80, 73, 70, 80, 81, 71, 86, 91, 84, 72, 103, 116, 117, 86, 91, 77, 99, 71, 71, 89, 87, 73, 85, 73, 109, 67, 72, 86]
82.6086956521739
Unit-Test: ArtikelId    82.608696
dtype: float64.


## Beste Ausgabe

In [7]:
def edition_with_highest_value(df, date_col="Datum"):
    start_val = 0
    best_day = ""
    all_dates = df[date_col].unique()
    for date in all_dates:
        mean_average = df[df[date_col] == date]["LW"].mean()
        if mean_average > start_val:
            start_val = mean_average
            best_day = date
    return best_day, start_val


best_day, lesewert = edition_with_highest_value(df)
print(f"Bester Tag: {best_day}, Wert: {round(lesewert, 2)}")

Bester Tag: 14.04.2022, Wert: 24.17


## Leser nach Uhrzeit

In [7]:
df_scans["Stunde"] = df_scans["Erfassungsuhrzeit"].str[:2].astype(int)
df_group = df_scans.groupby("Stunde", as_index=False)

all_hours = list(range(24))
all_cols = all_hours.copy()
all_cols.insert(0, "Datum")
all_cols

all_date = df_scans.Erscheinungsdatum.unique()

from collections import defaultdict
new_df = pd.DataFrame(columns=all_cols)

for dat in all_date:
    new_dict = defaultdict(int)
    df_ = df_scans[df_scans["Erscheinungsdatum"] == dat]
    new_dict["Datum"] = dat
    for hour in all_hours:
        df__ = df_[df_["Stunde"] == hour].WellenteilnahmenId.nunique()
        new_dict[hour] = df__

    new_ser = pd.Series(new_dict)

    new_df = new_df.append(new_ser, ignore_index=True)
    
nn = new_df.mean()
df_fin = pd.DataFrame([nn]).transpose().reset_index()
df_fin.columns = ["Uhrzeit", "Leser"]

<ipython-input-7-221349b081a3>:26: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  nn = new_df.mean()


In [13]:
df_fin.to_clipboard(decimal=",")